In [ ]:
import numpy as np
import tensorflow as tf

from tensorflow.keras.layers import Conv2D, Dense, Flatten, MaxPooling2D, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam
from art.estimators.classification import TensorFlowV2Classifier
from art.utils import load_mnist
import matplotlib.pyplot as plt
from art.attacks.extraction.copycat_cnn import CopycatCNN
from art.attacks.evasion import FastGradientMethod
from art.attacks.evasion.carlini import CarliniL2Method
from art.utils import random_targets
from art.defences.trainer.adversarial_trainer import AdversarialTrainer

In [ ]:
(X_train, y_train), (X_test, y_test), min_pixel_value, max_pixel_value = load_mnist()

nb_classes = 10

In [ ]:
model = Sequential()
model.add(Conv2D(1 , kernel_size=(7, 7), activation='relu'))
model.add(MaxPooling2D(pool_size=(4, 4)))
model.add(Flatten())
model.add(Dense(nb_classes, activation='softmax'))
model.compile()

model.summary()

In [ ]:
victim_classifier = TensorFlowV2Classifier(model=model, loss_object=tf.keras.losses.CategoricalCrossentropy(from_logits=False), optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), clip_values=(0, 1), nb_classes=nb_classes, input_shape=(28, 28, 1))
victim_classifier.fit(X_train, y_train, batch_size=128, nb_epochs=5)

In [ ]:
predictions = victim_classifier.predict(X_test)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print("Accuracy on benign test examples: {} %".format(accuracy * 100))


In [ ]:
target_index = 98
target_X = X_test[np.newaxis ,target_index]
target_pred = np.argmax(victim_classifier.predict(target_X), axis=1)[0]
plt.imshow(X_test[target_index].reshape(28, 28), cmap='gray')
print(f'Prediction {target_pred}, Ground Truth {np.argmax(y_test[target_index])}')

### Carlini & Wagner

In [ ]:
attack_num = 100 # 時間がかかるのでデータ数を絞る
carlini_wagner_attack = CarliniL2Method(classifier=victim_classifier, targeted=True, max_iter=10)
params = {'y': random_targets(y_test[:attack_num], victim_classifier.nb_classes)}

X_test_adv = carlini_wagner_attack.generate(x=X_test[:attack_num], **params)

preds = victim_classifier.predict(X_test_adv)
accuracy = np.sum(np.argmax(preds, axis=1) == np.argmax(y_test[:attack_num], axis=1)) / attack_num
print('Accuracy on adversarial test examples: {} %'.format(accuracy * 100))

In [ ]:
target_index = 23 # 0~99
target_X = X_test[np.newaxis ,target_index]
target_X_adv = X_test_adv[np.newaxis ,target_index]

pred_target = np.argmax(victim_classifier.predict(target_X), axis=1)[0]
pred_target_adv = np.argmax(victim_classifier.predict(target_X_adv), axis=1)[0]

fig, ax = plt.subplots(nrows=1, ncols=2)
ax[0].imshow(target_X.reshape(28, 28), cmap='gray')
ax[0].set_title(f'target Prediction: {pred_target}')
ax[1].imshow(target_X_adv.reshape(28, 28), cmap='gray')
ax[1].set_title(f'target adv Prediction: {pred_target_adv}')